# 画像収集や画像拡張系のpyファイル
- ディレクトリ構造  
.(groupwork)  
∟util.py  
  ∟本ファイル.py  
  ∟拡張元データimg(.jpg)&xmlファイル

- GoogleColabPRO(ランタイム：None, 標準)で実行

## 各ライブラリのver

In [3]:
!pip install albumentations==0.4.5
!pip install icrawler

     |████████████████████████████████| 122kB 2.7MB/s 
     |████████████████████████████████| 634kB 7.9MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=64378 sha256=8259acae337e4ab3c48b1f254fe8e8bf754d45df647c8e25cfaa46ec0ea65e8e
  Stored in directory: /root/.cache/pip/wheels/f0/a0/61/e50f93165a5ec7e7f5d65064e513239505bc4c06d2289557d3
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654021 sha256=1d946cf630d090bc1f8d22912e07ff0c21989ed3f5970f48fe2ae176d4498a76
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built albumentations imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [1]:
# 自分のマイドライブにマウントする

from google.colab import drive
drive.mount('/content/drive/')

# カレントディレクトリの変更
import os
os.chdir('/content/drive/My Drive/DIC')

# カレントディレクトリの取得
print(os.getcwd())

Mounted at /content/drive/
/content/drive/My Drive/DIC


## 1.ライブラリのimport

In [4]:
import numpy  as np
import os
import glob 
import re

import matplotlib.pyplot as plt

from PIL import Image

%matplotlib inline

import tensorflow
import cv2
import random

import icrawler

import albumentations as A
import xml.etree.ElementTree as ET

In [6]:
print("albumentations_version:{}".format(A.__version__))
print("icrawler_version:{}".format(icrawler.__version__))
print("tensorflow_version:{}".format(tensorflow.__version__))
print("cv2_version:{}".format(cv2.__version__))


albumentations_version:0.4.5
icrawler_version:0.6.3
tensorflow_version:2.3.0
cv2_version:4.1.2


In [ ]:
# ディレクトリpath配下のimg, xmlのリスト作成関数
def make_list(path):
  img_list = sorted(glob.glob(path + '/*' + ".jpg"))
  xml_list = sorted(glob.glob(path + '/*' + ".xml"))

  return img_list, xml_list

In [ ]:
# xmlからcoco形式のxmin, ymin, width, heights計算関数
def calc_bb(xml, num_bb=1):
    """
    format = yolo3 or yolo4 で指定
    yolo3 => [xmin, ymin, width, heights]
    yolo4 => [xcenter, ycenter, width, heights]

    bb = BoundingBox数
    return -> {"おーいお茶": [xmin, ymin, width, heights]}
    """
    with open(xml) as x:
        tree = ET.parse(xml)
        root = tree.getroot()
        bb_dic = {}
        for j in range(6, 6+num_bb):
            bb_name = root[j][0].text
            xmin = int(root[j][4][0].text)
            ymin = int(root[j][4][1].text)
            width = int(root[j][4][2].text) - xmin
            heights = int(root[j][4][3].text) - ymin
            bb_dic[bb_name] = [xmin, ymin, width, heights]

    return bb_dic

In [ ]:
# https://github.com/albumentations-team/albumentations_examples/blob/master/notebooks/example_bboxes.ipynb
# albumentations関数
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)
   
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, category_ids, category_id_to_name):
    # ここだけ変更
    if type(image) == str:
      image = cv2.imread(image)
      # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)
    # print(image)

## ↓ここから実行

In [ ]:
# bboxをcoco形式からyolo形式に変換する関数
def from_coco_to_yolo(bboxes, format="yolo3"):
    yolo_boxes = []
    for i, bbox in enumerate(bboxes):
        if format=="yolo3":
            xmax = bbox[0] + bbox[2]
            ymax = bbox[1] + bbox[3]
            yolo_boxes.append((int(bboxes[i][0]), int(bboxes[i][1]), int(xmax), int(ymax)))
        
        elif format=="yolo4":
            xcenter = bbox[0] + bbox[2] / 2
            ycenter = bbox[1] + bbox[3] / 2
            yolo_boxes.append((round(xcenter/300, 6),
                            round(ycenter/400, 6),
                            round(bbox[2]/300, 6), 
                            round(bbox[3]/400, 6)))
    
    return yolo_boxes

In [ ]:
# def annotation_write(tmp_bboxes, img, aug_num=None):
#   with open(os.getcwd()+os.sep+"annotation.txt", "a", encoding="UTF-8") as f:
#     # 画像パス書き込み
#     if aug_num==None:
#       f.write(str(img[:])+ " ")
#     else:
#       f.write(str(img[:-4]) + "_aug" + str(aug_num) +".jpg" + " ")
#     # bboxの書き込み "xmin, ymin, xmax, ymax, label"
#     for i, bbox in enumerate(tmp_bboxes):
#       f.write(",".join(map(str, bbox)))
#       f.write(","+str(i)+" ")
#     # 全ての書き込みが終了したら改行
#     f.write("\n")

In [ ]:
# def annotation_write_yolo3(tmp_bboxes, img, aug_num=None, id=None):
#     with open(os.getcwd()+os.sep+"annotation.txt", "a", encoding="UTF-8") as f:
#         # 画像パス書き込み
#         if aug_num==None:
#         f.write(str(img[:])+ " ")
#         else:
#         f.write(str(img[:-4]) + "_aug" + str(aug_num) +".jpg" + " ")
#         # bboxの書き込み "xmin, ymin, xmax, ymax, label"
#         for i, bbox in enumerate(tmp_bboxes):
#         f.write(",".join(map(str, bbox)))
#         f.write(","+str(id)+" ")
#         # 全ての書き込みが終了したら改行
#         f.write("\n")

In [ ]:
def annotation_write_yolo4(tmp_bboxes, img, aug_num=None, id=None):
    with open(os.getcwd()+os.sep+"annotation.txt", "a", encoding="UTF-8") as f:
        # 画像パス書き込み
        if aug_num==None:
            f.write(str(img[12:])+ " ")
        else:
            f.write(str(img[12:-4]) + "_aug" + str(aug_num) +".jpg" + " ")
        # bboxの書き込み "label, xcenter, ycenter, width, heights"
        for i, bbox in enumerate(tmp_bboxes):
            if id == None:
                f.write(str(i)+",")
            else:
                f.write(str(id)+",")

            tmp_str = ""
            for value in bbox:
                tmp_str += "{:.6f}".format(value) +","

            f.write(tmp_str[:-1]+" ")
            # f.write(",".join(map(str, bbox)))
            # f.write(map)
            # f.write(","+str(id)+" ")
        
        # 全ての書き込みが終了したら改行
        f.write("\n")

In [ ]:
# # データ拡張とannotation.txt作成関数
# # 保存先：拡張ｉｍｇ->元img同ディレクトリ annotation.txt->カレントディレクトリ
# for img, xml in zip(img_list, xml_list):
#   bb_dic = calc_bb2(xml)
#   bboxes = list(bb_dic.values())
#   category_ids = [1, 2, 3, 4, 5]
#   image = cv2.imread(img)
#   category_id_to_name = {1: 'おーいお茶', 2: 'コーラ', 3: 'グレープフルーツスカッシュ', 4: '午後の紅茶', 5: 'ポカリスエット'}
#   print("-----processing-----")

#   tmp = from_coco_to_yolo(bboxes)

#   # 元imgファイルのannotationテキストに書き込み
#   annotation_write2(tmp, img, aug_num=None, id=id)  
  
# #   反転
#   random.seed(7)
#   transform1 = A.Compose(
#       [A.HorizontalFlip(p=0.5)],
#       bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   transformed1 = transform1(image=image, bboxes=bboxes, category_ids=category_ids)
  
#   # 画像保存
#   cv2.imwrite(str(img[: -4]) + "_aug1.jpg", transformed1["image"])

#   tmp = from_coco_to_yolo(transformed1["bboxes"])
#   annotation_write(tmp, img, 1)

#   # 傾斜1
#   transform2 = A.Compose(
#       [A.ShiftScaleRotate(p=0.5)],
#       bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   transformed2 = transform2(image=image, bboxes=bboxes, category_ids=category_ids)
  
#   # 画像保存
#   cv2.imwrite(str(img[: -4]) + "_aug2.jpg", transformed2["image"])
  
#   tmp = from_coco_to_yolo(transformed2["bboxes"])
#   annotation_write(tmp, img, 2)  
  
#   # 反転/傾斜2
#   transform3 = A.Compose(
#       [A.HorizontalFlip(p=0.5), A.ShiftScaleRotate(p=0.25)],
#       bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   transformed3 = transform3(image=image, bboxes=bboxes, category_ids=category_ids)
  
#   # 画像保存
#   cv2.imwrite(str(img[: -4]) + "_aug3.jpg", transformed3["image"])
  
#   tmp = from_coco_to_yolo(transformed3["bboxes"])
#   annotation_write(tmp, img, 3)
  
#   # 傾斜3
#   transform4 = A.Compose(
#       [A.ShiftScaleRotate(p=0.75)],
#       bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   transformed4 = transform4(image=image, bboxes=bboxes, category_ids=category_ids)
  
#   # 画像保存
#   cv2.imwrite(str(img[: -4]) + "_aug4.jpg", transformed4["image"])
  
#   tmp = from_coco_to_yolo(transformed4["bboxes"])
#   annotation_write(tmp, img, 4)
  
#   # 反転させて輝度
#   transform5 = A.Compose([
#       A.HorizontalFlip(p=0.5),
#       A.ShiftScaleRotate(p=0.5),
#       A.RandomBrightnessContrast(p=0.3),
#       A.RGBShift(r_shift_limit=30, g_shift_limit=30, b_shift_limit=30, p=0.3),
#     ],bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   transformed5 = transform5(image=image, bboxes=bboxes, category_ids=category_ids)
  
#   # 画像保存
#   cv2.imwrite(str(img[: -4]) + "_aug5.jpg", transformed5["image"])
  
#   tmp = from_coco_to_yolo(transformed5["bboxes"])
#   annotation_write(tmp, img, 5)
  
#   # 中心に寄せる
#   transform6 = A.Compose(
#     [A.CenterCrop(height=20, width=20, p=1)],
#     bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   transformed6 = transform6(image=image, bboxes=bboxes, category_ids=category_ids)
#   cv2.imwrite(str(img[: -4]) + "_aug6.jpg", transformed6["image"])

#   tmp = from_coco_to_yolo(transformed6["bboxes"])
#   annotation_write2(tmp, img, 6, id=id)
#   # # ランダムシード
#   # # 反転
#   # random.seed(100)
#   # transform6 = A.Compose(
#   #     [A.HorizontalFlip(p=0.5)],
#   #     bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   # transformed6 = transform6(image=image, bboxes=bboxes, category_ids=category_ids)
#   # cv2.imwrite(str(img[:-4]) + "_aug6.jpg", transformed6["image"])
#   # with open(os.getcwd()+os.sep+"annotation.txt", "a", encoding="UTF-8") as f:
#   #   f.write(str(img[:-4]) + "_aug6.jpg" + " ")
#   #   for i in range(len(transformed6['bboxes'])):
#   #     f.write(",".join(map(str, map(int, transformed6['bboxes'][i]))))
#   #     f.write(","+str(i)+" ")
#   #   f.write("\n")
#   # # 傾斜
#   # transform7 = A.Compose(
#   #     [A.ShiftScaleRotate(p=0.5)],
#   #     bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   # transformed7 = transform7(image=image, bboxes=bboxes, category_ids=category_ids)
#   # cv2.imwrite(str(img[: -4]) + "_aug7.jpg", transformed7["image"])
#   # with open(os.getcwd()+os.sep+"annotation.txt", "a", encoding="UTF-8") as f:
#   #   f.write(str(img[: -4]) + "_aug7.jpg" + " ")
#   #   for i in range(len(transformed7['bboxes'])):
#   #     f.write(",".join(map(str, map(int, transformed7['bboxes'][i]))))
#   #     f.write(","+str(i)+" ")
#   #   f.write("\n")

#   # # 輝度変更2枚
#   # transform8 = A.Compose([
#   #         A.RandomBrightnessContrast(p=0.3),
#   #         A.RGBShift(r_shift_limit=30, g_shift_limit=30, b_shift_limit=30, p=0.3),
#   #     ],
#   #     bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   # transformed8 = transform8(image=image, bboxes=bboxes, category_ids=category_ids)
#   # cv2.imwrite(str(img[: -4]) + "_aug8.jpg", transformed8["image"])
#   # with open(os.getcwd()+os.sep+"annotation.txt", "a", encoding="UTF-8") as f:
#   #   f.write(str(img[: -4]) + "_aug8.jpg" + " ")
#   #   for i in range(len(transformed8['bboxes'])):
#   #     f.write(",".join(map(str, map(int, transformed8['bboxes'][i]))))
#   #     f.write(","+str(i)+" ")
#   #   f.write("\n")

#   # # 輝度変更 緑
#   # transform9 = A.Compose([
#   #         A.RandomBrightnessContrast(p=0.6),
#   #         A.RGBShift(r_shift_limit=100, g_shift_limit=100, b_shift_limit=100, p=1.0),
#   #     ],
#   #     bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   # transformed9 = transform9(image=image, bboxes=bboxes, category_ids=category_ids)
#   # cv2.imwrite(str(img[: -4]) + "_aug9.jpg", transformed9["image"])
#   # with open(os.getcwd()+os.sep+"annotation.txt", "a", encoding="UTF-8") as f:
#   #   f.write(str(img[: -4]) + "_aug9.jpg" + " ")
#   #   for i in range(len(transformed9['bboxes'])):
#   #     f.write(",".join(map(str, map(int, transformed9['bboxes'][i]))))
#   #     f.write(","+str(i)+" ")
#   #   f.write("\n")

#   # # 中心に寄せる
#   # transform10 = A.Compose(
#   #     [A.HorizontalFlip(p=0.1)],
#   #     bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   # transformed10 = transform5(image=image, bboxes=bboxes, category_ids=category_ids)
#   # cv2.imwrite(str(img[: -4]) + "_aug10.jpg", transformed10["image"])
#   # with open(os.getcwd()+os.sep+"annotation.txt", "a", encoding="UTF-8") as f:
#   #   f.write(str(img[: -4]) + "_aug10.jpg" + " ")
#   #   for i in range(len(transformed10['bboxes'])):
#   #     f.write(",".join(map(str, map(int, transformed10['bboxes'][i]))))
#   #     f.write(","+str(i)+" ")
#   #   f.write("\n")

-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----


In [ ]:
# # データ拡張とannotation.txt作成関数
# # 保存先：拡張ｉｍｇ->元img同ディレクトリ annotation.txt->カレントディレクトリ
# for img, xml in zip(img_list, xml_list):
#   bb_dic = calc_bb2(xml)
#   bboxes = list(bb_dic.values())
#   category_ids = [0, 1, 2, 3, 4, 5]
#   image = cv2.imread(img)
#   category_id_to_name = {0: 'hey_tea', 1: 'cola', 2: 'grapefruits', 3: 'gogo_tea', 4: 'pocari', 5:"calpis"}
#   if "hey_tea" in img:
#     id = 0
#   elif "cola" in img:
#     id = 1
#   elif "grapefruits" in img:
#     id = 2
#   elif "gogo_tea" in img:
#     id = 3
#   elif "pocari" in img:
#     id = 4
#   elif "calpis" in img:
#     id = 5

#   print("-----processing-----")

#   tmp = from_coco_to_yolo(bboxes)

#   # 元imgファイルのannotationテキストに書き込み
#   annotation_write2(tmp, img, aug_num=None, id=id)  
  
#   # 反転
#   random.seed(7)
#   transform1 = A.Compose(
#       [A.HorizontalFlip(p=0.5)],
#       bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   transformed1 = transform1(image=image, bboxes=bboxes, category_ids=category_ids)
  
#   # 画像保存
#   cv2.imwrite(str(img[: -4]) + "_aug1.jpg", transformed1["image"])

#   tmp = from_coco_to_yolo(transformed1["bboxes"])
#   annotation_write2(tmp, img, 1, id=id)

#   # 傾斜1
#   transform2 = A.Compose(
#       [A.ShiftScaleRotate(p=0.5)],
#       bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   transformed2 = transform2(image=image, bboxes=bboxes, category_ids=category_ids)
  
#   # 画像保存
#   cv2.imwrite(str(img[: -4]) + "_aug2.jpg", transformed2["image"])
  
#   tmp = from_coco_to_yolo(transformed2["bboxes"])
#   annotation_write2(tmp, img, 2, id=id)  
  
#   # 反転/傾斜2
#   transform3 = A.Compose(
#       [A.ShiftScaleRotate(p=0.1)],
#       bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   transformed3 = transform3(image=image, bboxes=bboxes, category_ids=category_ids)
  
#   # 画像保存
#   cv2.imwrite(str(img[: -4]) + "_aug3.jpg", transformed3["image"])
  
#   tmp = from_coco_to_yolo(transformed3["bboxes"])
#   annotation_write2(tmp, img, 3, id=id)
  
#   # 反転させて輝度
#   transform4 = A.Compose([
#       A.HorizontalFlip(p=0.5),
#       A.ShiftScaleRotate(p=0.5),
#       A.RandomBrightnessContrast(p=0.3),
#       A.RGBShift(r_shift_limit=30, g_shift_limit=30, b_shift_limit=30, p=0.3),
#     ],bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   transformed4 = transform4(image=image, bboxes=bboxes, category_ids=category_ids)
  
#   # 画像保存
#   cv2.imwrite(str(img[: -4]) + "_aug4.jpg", transformed4["image"])
  
#   tmp = from_coco_to_yolo(transformed4["bboxes"])
#   annotation_write2(tmp, img, 4, id=id)
  
#   # 中心に寄せる
#   transform5 = A.Compose(
#     [A.CenterCrop(height=300, width=250, p=1)],
#     bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   transformed5 = transform5(image=image, bboxes=bboxes, category_ids=category_ids)
#   cv2.imwrite(str(img[: -4]) + "_aug5.jpg", transformed5["image"])

#   tmp = from_coco_to_yolo(transformed5["bboxes"])
#   annotation_write2(tmp, img, 5, id=id)

#   # 中心に寄せる
#   transform6 = A.Compose(
#     [A.CenterCrop(height=250, width=200, p=1)],
#     bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   transformed6 = transform6(image=image, bboxes=bboxes, category_ids=category_ids)
#   cv2.imwrite(str(img[: -4]) + "_aug6.jpg", transformed6["image"])

#   tmp = from_coco_to_yolo(transformed6["bboxes"])
#   annotation_write2(tmp, img, 6, id=id)

-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processi

In [ ]:
# # データ拡張とannotation.txt作成関数
# # 保存先：拡張ｉｍｇ->元img同ディレクトリ annotation.txt->カレントディレクトリ
# for img, xml in zip(img_list, xml_list):
#   bb_dic = calc_bb2(xml)
#   bboxes = list(bb_dic.values())
#   category_ids = [0, 1, 2, 3, 4]
#   image = cv2.imread(img)
#   category_id_to_name = {0: 'hey_tea', 1: 'cola', 2: 'grapefruits', 3: 'gogo_tea', 4: 'pocari'}
#   if "hey_tea" in img:
#     id = 0
#   elif "cola" in img:
#     id = 1
#   elif "grapefruits" in img:
#     id = 2
#   elif "gogo_tea" in img:
#     id = 3
#   elif "pocari" in img:
#     id = 4

#   print("-----processing-----")

#   tmp = from_coco_to_yolo4(bboxes)

#   # 元imgファイルのannotationテキストに書き込み
#   annotation_write_yolo(tmp, img, aug_num=None, id=id)

#   for i in range(10):
#     p1 = np.random.random()
    
#     # 輝度の変更（ランダム）
#     transform7 = A.Compose([
#         A.RandomBrightnessContrast(p=p1),
#         ],bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#     transformed7 = transform7(image=image, bboxes=bboxes, category_ids=category_ids)

#     # 画像保存
#     cv2.imwrite(str(img[: -4]) + "_aug" + str(i) +".jpg", transformed7["image"])
    
#     tmp = from_coco_to_yolo(transformed7["bboxes"])
#     annotation_write2(tmp, img, i, id=id)

-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processing-----
-----processi

In [ ]:
path = "./training4"

img_list, xml_list = make_list(path)

print(img_list)
print(xml_list)

print(len(img_list))
print(len(xml_list))

['./training4/data1.jpg', './training4/data10.jpg', './training4/data11.jpg', './training4/data2.jpg', './training4/data3.jpg', './training4/data4.jpg', './training4/data5.jpg', './training4/data6.jpg', './training4/data7.jpg', './training4/data8.jpg', './training4/data9.jpg']
['./training4/data1.xml', './training4/data10.xml', './training4/data11.xml', './training4/data2.xml', './training4/data3.xml', './training4/data4.xml', './training4/data5.xml', './training4/data6.xml', './training4/data7.xml', './training4/data8.xml', './training4/data9.xml']
11
11


In [ ]:
"""yolo4"""

# データ拡張とannotation.txt作成関数
# 保存先：拡張ｉｍｇ->元img同ディレクトリ annotation.txt->カレントディレクトリ
for epoch, (img, xml) in enumerate(zip(img_list, xml_list)):
  bb_dic = calc_bb(xml)
  bboxes = list(bb_dic.values())
  category_ids = [1, 2, 3, 4, 5]
  image = cv2.imread(img)
  category_id_to_name = {1: 'おーいお茶', 2: 'コーラ', 3: 'グレープフルーツスカッシュ', 4: '午後の紅茶', 5: 'ポカリスエット'}
  print("---loop{}--processing-----".format(str(epoch)))

  tmp = from_coco_to_yolo4(bboxes)
#   print("base", tmp)
  # 元imgファイルのannotationテキストに書き込み
  annotation_write_yolo4(tmp, img, aug_num=None)  
  
  for i in range(10):
    p1 = np.random.random()
    
    # 輝度の変更（ランダム）
    transform7 = A.Compose([
        A.RandomBrightnessContrast(p=p1),
        ],bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
    transformed7 = transform7(image=image, bboxes=bboxes, category_ids=category_ids)

    # 画像保存
    cv2.imwrite(str(img[: -4]) + "_aug" + str(i) +".jpg", transformed7["image"])
    
    tmp = from_coco_to_yolo4(transformed7["bboxes"])
    # print("aug", tmp)
    annotation_write_yolo4(tmp, img, i)

---loop0--processing-----
base [(0.15, 0.45875, 0.18, 0.3875), (0.33, 0.465, 0.186667, 0.385), (0.495, 0.48125, 0.17, 0.3625), (0.663333, 0.4825, 0.173333, 0.365), (0.835, 0.49, 0.176667, 0.36)]
aug [(0.15, 0.45875, 0.18, 0.3875), (0.33, 0.465, 0.186667, 0.385), (0.495, 0.48125, 0.17, 0.3625), (0.663333, 0.4825, 0.173333, 0.365), (0.835, 0.49, 0.176667, 0.36)]
aug [(0.15, 0.45875, 0.18, 0.3875), (0.33, 0.465, 0.186667, 0.385), (0.495, 0.48125, 0.17, 0.3625), (0.663333, 0.4825, 0.173333, 0.365), (0.835, 0.49, 0.176667, 0.36)]
aug [(0.15, 0.45875, 0.18, 0.3875), (0.33, 0.465, 0.186667, 0.385), (0.495, 0.48125, 0.17, 0.3625), (0.663333, 0.4825, 0.173333, 0.365), (0.835, 0.49, 0.176667, 0.36)]
aug [(0.15, 0.45875, 0.18, 0.3875), (0.33, 0.465, 0.186667, 0.385), (0.495, 0.48125, 0.17, 0.3625), (0.663333, 0.4825, 0.173333, 0.365), (0.835, 0.49, 0.176667, 0.36)]
aug [(0.15, 0.45875, 0.18, 0.3875), (0.33, 0.465, 0.186667, 0.385), (0.495, 0.48125, 0.17, 0.3625), (0.663333, 0.4825, 0.173333, 0.36

In [ ]:
path = "./training9"

img_list, xml_list = make_list(path)

print(img_list)
print(xml_list)

print(len(img_list))
print(len(xml_list))

['./training9/cola_1.jpg', './training9/cola_10.jpg', './training9/cola_11.jpg', './training9/cola_12.jpg', './training9/cola_13.jpg', './training9/cola_14.jpg', './training9/cola_15.jpg', './training9/cola_16.jpg', './training9/cola_2.jpg', './training9/cola_3.jpg', './training9/cola_4.jpg', './training9/cola_5.jpg', './training9/cola_6.jpg', './training9/cola_7.jpg', './training9/cola_8.jpg', './training9/cola_9.jpg', './training9/gogo_tea_1.jpg', './training9/gogo_tea_10.jpg', './training9/gogo_tea_11.jpg', './training9/gogo_tea_12.jpg', './training9/gogo_tea_13.jpg', './training9/gogo_tea_14.jpg', './training9/gogo_tea_15.jpg', './training9/gogo_tea_16.jpg', './training9/gogo_tea_2.jpg', './training9/gogo_tea_3.jpg', './training9/gogo_tea_4.jpg', './training9/gogo_tea_5.jpg', './training9/gogo_tea_6.jpg', './training9/gogo_tea_7.jpg', './training9/gogo_tea_8.jpg', './training9/gogo_tea_9.jpg', './training9/grapefruits_1.jpg', './training9/grapefruits_10.jpg', './training9/grapefrui

In [ ]:
# # yolo4データ

# # データ拡張とannotation.txt作成関数
# # 保存先：拡張ｉｍｇ->元img同ディレクトリ annotation.txt->カレントディレクトリ
# for epoch, (img, xml) in enumerate(zip(img_list, xml_list)):
#   bb_dic = calc_bb2(xml)
#   bboxes = list(bb_dic.values())
#   category_ids = [0, 1, 2, 3, 4]
#   image = cv2.imread(img)
#   category_id_to_name = {0: 'hey_tea', 1: 'cola', 2: 'grapefruits', 3: 'gogo_tea', 4: 'pocari'}
#   if "hey_tea" in img:
#     id = 0
#   elif "cola" in img:
#     id = 1
#   elif "grapefruits" in img:
#     id = 2
#   elif "gogo_tea" in img:
#     id = 3
#   elif "pocari" in img:
#     id = 4

#   print("---loop{}--processing-----".format(str(epoch)))

#   tmp = from_coco_to_yolo4(bboxes)

#   # 元imgファイルのannotationテキストに書き込み
#   annotation_write_yolo4(tmp, img, aug_num=None, id=id)

#   for i in range(10):
#     p1 = np.random.random()
    
#     # 輝度の変更（ランダム）
#     transform7 = A.Compose([
#         A.RandomBrightnessContrast(p=p1),
#         ],bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#     transformed7 = transform7(image=image, bboxes=bboxes, category_ids=category_ids)

#     # 画像保存
#     cv2.imwrite(str(img[: -4]) + "_aug" + str(i) +".jpg", transformed7["image"])
    
#     tmp = from_coco_to_yolo4(transformed7["bboxes"])
#     annotation_write_yolo4(tmp, img, i, id=id)
  

#     # 中心に寄せる
#   transform5 = A.Compose(
#     [A.CenterCrop(height=320, width=240, p=1)],
#     bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   transformed5 = transform5(image=image, bboxes=bboxes, category_ids=category_ids)
#   cv2.imwrite(str(img[: -4]) + "_aug10.jpg", transformed5["image"])

#   tmp = from_coco_to_yolo4(transformed5["bboxes"])
#   annotation_write_yolo4(tmp, img, 10, id=id)

#   # 中心に寄せる
#   transform6 = A.Compose(
#     [A.CenterCrop(height=280, width=210, p=1)],
#     bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),)
#   transformed6 = transform6(image=image, bboxes=bboxes, category_ids=category_ids)
#   cv2.imwrite(str(img[: -4]) + "_aug11.jpg", transformed6["image"])

#   tmp = from_coco_to_yolo4(transformed6["bboxes"])
#   annotation_write_yolo4(tmp, img, 11, id=id)

---loop0--processing-----
---loop1--processing-----
---loop2--processing-----
---loop3--processing-----
---loop4--processing-----
---loop5--processing-----
---loop6--processing-----
---loop7--processing-----
---loop8--processing-----
---loop9--processing-----
---loop10--processing-----
---loop11--processing-----
---loop12--processing-----
---loop13--processing-----
---loop14--processing-----
---loop15--processing-----
---loop16--processing-----
---loop17--processing-----
---loop18--processing-----
---loop19--processing-----
---loop20--processing-----
---loop21--processing-----
---loop22--processing-----
---loop23--processing-----
---loop24--processing-----
---loop25--processing-----
---loop26--processing-----
---loop27--processing-----
---loop28--processing-----
---loop29--processing-----
---loop30--processing-----
---loop31--processing-----
---loop32--processing-----
---loop33--processing-----
---loop34--processing-----
---loop35--processing-----
---loop36--processing-----
---loop37--

In [ ]:
from icrawler.builtin import BingImageCrawler
import sys
import os

# argv = sys.argv
argv1 = ["./綾鷹", "./アクエリアス", "./午後の紅茶 ストレートティ", "./アクエリアスビタミン", "./クラフトボス ブラウン"]
# 保存ディレクトリが存在していなければ作る

    for arg in argv1:
        if not os.path.isdir(arg):
            os.makedirs(arg)
        crawler = BingImageCrawler(storage={"root_dir": arg})
        crawler.crawl(keyword=arg[2:], max_num=50)

2020-10-18 02:43:07,278 - INFO - icrawler.crawler - start crawling...
2020-10-18 02:43:07,280 - INFO - icrawler.crawler - starting 1 feeder threads...
2020-10-18 02:43:07,285 - INFO - feeder - thread feeder-001 exit
2020-10-18 02:43:07,285 - INFO - icrawler.crawler - starting 1 parser threads...
2020-10-18 02:43:07,293 - INFO - icrawler.crawler - starting 1 downloader threads...
2020-10-18 02:43:07,632 - INFO - parser - parsing result page https://www.bing.com/images/async?q=綾鷹&first=0
2020-10-18 02:43:07,930 - INFO - downloader - image #1	https://askul.c.yimg.jp/img/product/3L1/1363478_3L1.jpg
2020-10-18 02:43:08,050 - INFO - downloader - image #2	https://askul.c.yimg.jp/img/product/3L1/600907_3L1.jpg
2020-10-18 02:43:08,509 - INFO - downloader - image #3	http://photo.kenko.com/E373342H_L.jpg
2020-10-18 02:43:08,976 - INFO - downloader - image #4	https://mag.japaaan.com/wp-content/uploads/2017/03/660ayatakka.jpg
2020-10-18 02:43:09,097 - INFO - downloader - image #5	https://askul.c.yi